In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2 as cv
import rasterio
import os
import glob
import warnings
import random
import numpy as np
import time
# Suppress runtime warnings # HFDT-CONDA
warnings.filterwarnings("ignore", message="invalid value encountered in scalar divide")

In [13]:
from torch.utils.data import Dataset
from PIL import Image
import torch
import torch.nn as nn
import torchvision.transforms as transforms

In [14]:
from training_dataloader import CustomDataset
from model import CombinedModel
from torch.utils.data import DataLoader

In [15]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [16]:
dataset = CustomDataset(data_path=".././Test/GroundTruth/test_HIPS_HYBRIDS_2023_V2.3.csv", 
                 date_path=".././Test/GroundTruth/DateofCollection.xlsx",
                 mother_path=".././Test/",
                 is_testing=True,
                 device=device)
test_loader = DataLoader(dataset, batch_size=1, shuffle=False)

In [17]:
model_paths = []
model_paths.append("./saved_models/model_1.pth")
model_paths.append("./saved_models/model_2.pth")
model_paths.append("./saved_models/model_3.pth")
model_paths.append("./saved_models/model_4.pth")

In [18]:
models = [CombinedModel(device=device).to(device) for _ in model_paths]

for model, path in zip(models, model_paths):
    model.load_state_dict(torch.load(path))
    model.eval()

/work/flemingc/prajwal/me699/HF_DT/.conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/work/flemingc/prajwal/me699/HF_DT/.conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/work/flemingc/prajwal/me699/HF_DT/.conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:

test_yieldperacre = []

for i, val_sample in enumerate(test_loader):
    predictions = []
    
    for model in models:
        with torch.no_grad():
            this_out = model(val_sample[0], val_sample[2]['satelliteImages'].to(device), 
                val_sample[2]['satelliteImagesInfo'].to(device),
                val_sample[2]['satelliteImagesLen'].to(device)).item()*100+150
            predictions.append(this_out)
    
    ens_out = np.median(predictions)
    test_yieldperacre.append(ens_out)


In [20]:
test_yieldperacre

[185.7956811785698,
 159.03377905488014,
 149.91064853966236,
 148.61583728343248,
 162.61287406086922,
 174.87010285258293,
 177.76971235871315,
 171.78510650992393,
 145.32386921346188,
 174.68365728855133,
 154.79616709053516,
 176.33836939930916,
 145.92531621456146,
 136.76372841000557,
 153.62565610557795,
 145.44808194041252,
 173.225087672472,
 170.31367309391499,
 172.78972864151,
 146.34482488036156,
 85.33986508846283,
 171.84833139181137,
 201.2293040752411,
 128.39688286185265,
 131.4944639801979,
 157.44503736495972,
 164.90713134407997,
 185.7904240489006,
 164.9416022002697,
 160.6685133650899,
 179.4714331626892,
 138.88772428035736,
 156.65824972093105,
 150.8415326476097,
 172.66748026013374,
 159.65418070554733,
 160.62623150646687,
 187.4062865972519,
 115.8084973692894,
 130.9127874672413,
 193.90448480844498,
 201.45382434129715,
 164.30730670690536,
 189.5099326968193,
 184.6002921462059,
 139.33606445789337,
 137.77349032461643,
 181.35231733322144,
 142.754916

In [21]:
val_data_path = ".././Test/GroundTruth/test_HIPS_HYBRIDS_2023_V2.3.csv"
val_data = pd.read_csv(val_data_path)

In [22]:
val_data['yieldPerAcre'] = test_yieldperacre
new_val_data_path = ".././test_HIPS_HYBRIDS_2023_new.csv"
val_data.to_csv(new_val_data_path)